# Programming Project - Unit 3,3
*by Igor A. Brandão and Leandro Antonio Feliciano da Silva*

**Goals**
The purpose of this project is explore the following:

- Access Health Graph API - Runkeeper content;
- Full content of the statistical part seen in the course;
- Graphs generation;
- Geolocation analysis and hypotheses should be explained in detail;
- Web scraping.

<hr>

# Global Imports section

Import the necessary libraries to handle 

- Geocoding;
- Maps;
- File input;
- Heatmap;
- Bokeh charts;
- Numpy library;
- Tqdm progress bar
- Requests;
- urlopen;
- HTTPError;
- BeautifulSoup
- Regular expression

In [ ]:
### Library necessary to run this IPython Notebook
!pip install geocoder
!pip install folium
!pip install tqdm
!pip install tabulate

In [207]:
# Import pandas
import pandas as pd

# Import google geocoder
import geocoder as gc

# Import numpy library
import numpy as np

# Import folium heatmap
import folium
from folium.plugins import HeatMap

# Import tqdm progressing bar plugin
from tqdm import tqdm

# Import bokeh libraries
from bokeh.plotting import figure
from bokeh.charts import Bar, Histogram, Donut, BoxPlot, Line, output_notebook, show
from bokeh.layouts import row, gridplot, column
from bokeh.models import HoverTool
from bokeh.charts.attributes import cat, color
from bokeh.charts.operations import blend
from bokeh.models import HoverTool
from bokeh.charts.attributes import AttrSpec, ColorAttr, MarkerAttr, CatAttr

# Import request libraries
from urllib.request import Request, urlopen
from urllib.error import HTTPError

# Import web scraping libraries
from bs4 import BeautifulSoup
import re # regular expression

# Import API libraries
import requests
import json
from pandas.io.json import json_normalize

<hr>

# API section

## API data retrieving

#### In the cell bellow, we perform a connection with Health Graph API - Runkeeper.

In [39]:
# Access token
ACCESS_TOKEN = '25bc30d6dd6f4b99bbeb48e8619103b4'

# Base URI
api_URI = "http://api.runkeeper.com/fitnessActivities"

# Number of results
pageSize = 300

# Final URI
url = '%s?pageSize=%s&access_token=%s' % \
    (api_URI, pageSize, ACCESS_TOKEN,)

# print(url)

# Receive the results from API
api_content = requests.get(url).json()

print(json.dumps(api_content, indent=1))

{
 "size": 245,
 "items": [
  {
   "utc_offset": -3,
   "duration": 1037,
   "start_time": "Fri, 23 Jun 2017 12:40:11",
   "total_calories": 148,
   "tracking_mode": "outdoor",
   "total_distance": 5345.75479639154,
   "entry_mode": "API",
   "has_path": true,
   "source": "RunKeeper",
   "type": "Cycling",
   "uri": "/fitnessActivities/1005956310"
  },
  {
   "utc_offset": -3,
   "duration": 1140,
   "start_time": "Fri, 23 Jun 2017 07:50:54",
   "total_calories": 143,
   "tracking_mode": "outdoor",
   "total_distance": 6081.16390011933,
   "entry_mode": "API",
   "has_path": true,
   "source": "RunKeeper",
   "type": "Cycling",
   "uri": "/fitnessActivities/1005843765"
  },
  {
   "utc_offset": -3,
   "duration": 1285,
   "start_time": "Thu, 22 Jun 2017 12:41:16",
   "total_calories": 184,
   "tracking_mode": "outdoor",
   "total_distance": 6589.61628038915,
   "entry_mode": "API",
   "has_path": true,
   "source": "RunKeeper",
   "type": "Cycling",
   "uri": "/fitnessActivities/10055

## JSON to Data Frame conversion

In order to have a better data manipulation, in the next cell we perform a conversion of importe in json format from API to pandas data frame

In [63]:
# Perform a conversion from JSON to Data Frame
api_df = json_normalize(api_content['items'])

# Converts the duration from seconds to minutes
api_df['duration'] = api_df['duration']/60
api_df['duration'] = api_df['duration'].round(2);

api_df

,duration,entry_mode,has_path,source,start_time,total_calories,total_distance,tracking_mode,type,uri,utc_offset
0,17.28,API,True,RunKeeper,"Fri, 23 Jun 2017 12:40:11",148,5345.754796,outdoor,Cycling,/fitnessActivities/1005956310,-3
1,19.00,API,True,RunKeeper,"Fri, 23 Jun 2017 07:50:54",143,6081.163900,outdoor,Cycling,/fitnessActivities/1005843765,-3
2,21.42,API,True,RunKeeper,"Thu, 22 Jun 2017 12:41:16",184,6589.616280,outdoor,Cycling,/fitnessActivities/1005535586,-3
3,20.58,API,True,RunKeeper,"Thu, 22 Jun 2017 07:37:35",147,6162.944349,outdoor,Cycling,/fitnessActivities/1005303953,-3
4,19.30,API,True,RunKeeper,"Tue, 20 Jun 2017 12:35:21",156,5382.671145,outdoor,Cycling,/fitnessActivities/1004255363,-3
5,19.88,API,True,RunKeeper,"Tue, 20 Jun 2017 07:30:08",144,6086.986813,outdoor,Cycling,/fitnessActivities/1004088987,-3
6,17.02,API,True,RunKeeper,"Mon, 19 Jun 2017 12:36:20",141,5389.115056,outdoor,Cycling,/fitnessActivities/1003621881,-3
7,23.08,API,True,RunKeeper,"Mon, 19 Jun 2017 07:43:02",156,6143.735127,outdoor,Cycling,/fitnessActivities/1003493993,-3
8,50.43,API,True,RunKeeper,"Sat, 17 Jun 2017 14:47:50",310,8859.432268,outdoor,Cycling,/fitnessActivities/1002663916,-3
9,18.17,API,True,RunKeeper,"Wed, 14 Jun 2017 13:26:09",153,5753.597600,outdoor,Cycling,/fitnessActivities/1000879327,-3


## Data export [optional]

In order to visualize the data into an excel file, the cell bellow is responsible for exporting the data.

In [57]:
# Export the new dataSet to csv
api_df.to_csv('dataSource.csv', encoding="utf-8")

<hr>

# Statistic section

#### Here in this section, we'll handle the statistics infos.

#### The idea is to use the ***top-down analysis***, from the more generic context to the specific one

## 1) Exercises summary

## 2) Activites overall times

In [102]:
# =================================================================================
# Data selection
# =================================================================================

# =================================================================================
# Chart plotting
# =================================================================================

# Tools
TOOLS = 'box_zoom,box_select,crosshair,resize,reset,hover,save'

# Make a bar chart: p
p = Bar(api_df, values='duration', label='type', agg='mean', color='type',
            legend='bottom_right', background_fill_color="#E8DDCB",
            plot_width=750, plot_height=500, tools=TOOLS)

# Set the y and x axis label
p.yaxis.axis_label= 'Activity overall time'
p.xaxis.axis_label= 'Activity type'

# Set hover to bars
hover = p.select(dict(type=HoverTool))
hover.tooltips = [('Tipo de atividade', '@type'),('Average time:',' @height')]

# Configure visual properties on a plot's title attribute
p.title.text = "Overall time by activity type"
p.title.align = "center"
p.title.text_font_size = "25px"

# Call the output_notebook() 
output_notebook()
show(p)

Loading BokehJS ...

<hr>

## 2) Activites calories burning

In [105]:
# =================================================================================
# Data selection
# =================================================================================

# =================================================================================
# Chart plotting
# =================================================================================

# Tools
TOOLS = 'box_zoom,box_select,crosshair,resize,hover,reset,save'

# Make a bar chart: p
bar = Bar(api_df, values='total_calories', label='type', agg='sum', color='type',
            legend='bottom_right', background_fill_color="#E8DDCB",
            tools=TOOLS)

# Make a box plot: unit 1
box = BoxPlot(api_df, values='total_calories', label='type', color='type',
            legend='bottom_right', background_fill_color="#E8DDCB",
            tools=TOOLS)

# Set the y and x axis label
p.yaxis.axis_label= 'Activity total calories'
p.xaxis.axis_label= 'Activity type'

# Set hover to bars
hover = p.select(dict(type=HoverTool))
hover.tooltips = [('Tipo de atividade', '@type'),('Total calories:',' @height')]

# Configure visual properties on a plot's title attribute
bar.title.text = "Total calories burned by activity type"
bar.title.align = "center"
bar.title.text_font_size = "25px"

box.title.text = "Total calories burned by activity type"
box.title.align = "center"
box.title.text_font_size = "25px"

# Create a list containing plots
row1 = [bar,box]

# Create a gridplot using row1 and row2: layout
layout = gridplot([row1],sizing_mode='scale_width', plot_height=750)

# Call the output_notebook() 
output_notebook()
show(layout)

Loading BokehJS ...

<hr>

## 3) Activites by period

First of all we need to split the information by period. To achieve that, the idea is apply a group selection by partial string in timestamp column 

## Timestamp split into columns

In order to have a better way to handle the data by period, it'll be necessary split the timestamp column into separate columns

In [191]:
# =================================================================================
# Dataframe timestamp split
# =================================================================================

# Copy the data
data_by_period = api_df.copy()
data_by_period["month"] = 0
data_by_period["month_index"] = 0
data_by_period["year"] = 0

# Fill the years
data_by_period.loc[data_by_period['start_time'].str.contains('2017'), 'year'] = '2017'
data_by_period.loc[data_by_period['start_time'].str.contains('2016'), 'year'] = '2016'
data_by_period.loc[data_by_period['start_time'].str.contains('2015'), 'year'] = '2015'
data_by_period.loc[data_by_period['start_time'].str.contains('2014'), 'year'] = '2014'

# Fill the months
data_by_period.loc[data_by_period['start_time'].str.contains('Jan'), 'month'] = 'Jan'
data_by_period.loc[data_by_period['start_time'].str.contains('Jan'), 'month_index'] = '1'

data_by_period.loc[data_by_period['start_time'].str.contains('Feb'), 'month'] = 'Feb'
data_by_period.loc[data_by_period['start_time'].str.contains('Feb'), 'month_index'] = '2'

data_by_period.loc[data_by_period['start_time'].str.contains('Mar'), 'month'] = 'Mar'
data_by_period.loc[data_by_period['start_time'].str.contains('Mar'), 'month_index'] = '3'

data_by_period.loc[data_by_period['start_time'].str.contains('Apr'), 'month'] = 'Apr'
data_by_period.loc[data_by_period['start_time'].str.contains('Apr'), 'month_index'] = '4'

data_by_period.loc[data_by_period['start_time'].str.contains('May'), 'month'] = 'May'
data_by_period.loc[data_by_period['start_time'].str.contains('May'), 'month_index'] = '5'

data_by_period.loc[data_by_period['start_time'].str.contains('Jun'), 'month'] = 'Jun'
data_by_period.loc[data_by_period['start_time'].str.contains('Jun'), 'month_index'] = '6'

data_by_period.loc[data_by_period['start_time'].str.contains('Jul'), 'month'] = 'Jul'
data_by_period.loc[data_by_period['start_time'].str.contains('Jul'), 'month_index'] = '7'

data_by_period.loc[data_by_period['start_time'].str.contains('Aug'), 'month'] = 'Aug'
data_by_period.loc[data_by_period['start_time'].str.contains('Aug'), 'month_index'] = '8'

data_by_period.loc[data_by_period['start_time'].str.contains('Sep'), 'month'] = 'Sep'
data_by_period.loc[data_by_period['start_time'].str.contains('Sep'), 'month_index'] = '9'

data_by_period.loc[data_by_period['start_time'].str.contains('Oct-'), 'month'] = 'Oct'
data_by_period.loc[data_by_period['start_time'].str.contains('Oct'), 'month_index'] = '10'

data_by_period.loc[data_by_period['start_time'].str.contains('Nov'), 'month'] = 'Nov'
data_by_period.loc[data_by_period['start_time'].str.contains('Nov'), 'month_index'] = '11'

data_by_period.loc[data_by_period['start_time'].str.contains('Dec'), 'month'] = 'Dec'
data_by_period.loc[data_by_period['start_time'].str.contains('Dec'), 'month_index'] = '12'


## Data export [optional]

In order to avoid replacing the timestamp to other columns, we'll export the new dataSet to a csv file. You can skip this operation because the new .csv dataSet is already included in the project

In [192]:
# Export the new dataSet to csv
data_by_period.to_csv('dataSourceByPeriod.csv', encoding="utf-8")

## 2017 overview

In [208]:
# =================================================================================
# Data selection
# =================================================================================

# Import the fertility.csv data: data
data_by_period = pd.read_csv("dataSourceByPeriod.csv", encoding = 'latin2')

# Filter the activities by year
activities2017 = data_by_period[data_by_period["year"] == 2017].sort_values('month_index')

# =================================================================================
# Chart plotting
# =================================================================================

# Tools
TOOLS = 'box_zoom,box_select,crosshair,resize,hover,reset,save'

barMonth = Bar(activities2017, values='total_distance', label=CatAttr(columns=['month'], sort=False), color='month', agg='sum', 
                      legend='top_left', tools=TOOLS)

barCaloryMonth = Bar(activities2017, values='total_calories', label=CatAttr(columns=['month'], sort=False), color='month', agg='sum', 
                      legend='top_left', tools=TOOLS)

# Set the y and x axis label
barMonth.yaxis.axis_label= 'Total distance'
barMonth.xaxis.axis_label= 'Month'

barCaloryMonth.yaxis.axis_label= 'Total calories'
barCaloryMonth.xaxis.axis_label= 'Month'

# Set hover to bars
hover = barMonth.select(dict(type=HoverTool))
hover.tooltips = [('Month', '@x'),('Total distance:',' @height')]

hover = barCaloryMonth.select(dict(type=HoverTool))
hover.tooltips = [('Month', '@x'),('Total calories:',' @height')]

# Configure visual properties on a plot's title attribute
barMonth.title.text = "Total distance by month in 2017"
barMonth.title.align = "center"
barMonth.title.text_font_size = "25px"

barCaloryMonth.title.text = "Total calories by month in 2017"
barCaloryMonth.title.align = "center"
barCaloryMonth.title.text_font_size = "25px"

# Create a list containing plots
row1 = [barMonth, barCaloryMonth]

# Create a gridplot using row1 and row2: layout
layout = gridplot([row1],sizing_mode='scale_width', plot_height=650, plot_width=900)

# Call the output_notebook() 
output_notebook()
show(layout)

Loading BokehJS ...

## 5) Overall grades dispersion

In [50]:
# =================================================================================
# Data selection
# =================================================================================

# Select the subject list
sql = "select overall_grade from Grade where overall_grade IS NOT NULL and overall_grade <> '' \
        and overall_grade > 0.0"

# Execute query and store records in DataFrame: data
data = pd.read_sql_query(sql, engine)

# Convert the data into a dataFrame groupped values
grades = data.copy()
grades["Count"] = 0

# Group the overall grades and count the group frequency
grades = pd.DataFrame(grades.groupby(["overall_grade"])['Count'].count()).reset_index()

# =================================================================================
# Chart plotting
# =================================================================================

# Tools
TOOLS = 'box_zoom,box_select,crosshair,resize,reset,hover,save'

# Grades between 0.0 and 3.0
grades1 = grades[(0.0 < grades['overall_grade']) & (grades['overall_grade'] < 3.0)]['overall_grade']
frequency1 = grades[(0.0 < grades['overall_grade']) & (grades['overall_grade'] < 3.0)]["Count"]

# 3.1 <= Grades <= 5.0
grades2 = grades[(3.1 <= grades['overall_grade']) & (grades['overall_grade'] <= 5.0)]['overall_grade']
frequency2 = grades[(3.1 <= grades['overall_grade']) & (grades['overall_grade'] <= 5.0)]["Count"]

# 5.1 <= Grades <= 8.0
grades3 = grades[(5.1 <= grades['overall_grade']) & (grades['overall_grade'] <= 8.0)]['overall_grade']
frequency3 = grades[(5.1 <= grades['overall_grade']) & (grades['overall_grade'] <= 8.0)]["Count"]

# 8.1 < Grades
grades4 = grades[grades['overall_grade'] > 8.1]['overall_grade']
frequency4 = grades[grades['overall_grade'] > 8.1]["Count"]

# Create the figure: p
p = figure(x_axis_label='Grades', y_axis_label='Grades frequency', tools=TOOLS)

# Add the data to the plot
p.circle(grades1, frequency1, color="#990990", size=10, alpha=0.8)
p.circle(grades2, frequency2, color="#990000", size=10, alpha=0.8)
p.circle(grades3, frequency3, color="#009900", size=10, alpha=0.8)
p.circle(grades4, frequency4, color="#000099", size=10, alpha=0.8)

# Configure visual properties on a plot's title attribute
p.title.text = "Overall grades dispersion"
p.title.align = "center"
p.title.text_font_size = "25px"

# Call the output_notebook() 
output_notebook()

# Display the plot
show(p)

Loading BokehJS ...

<hr>

## 6) Overall grades historic chart

In [86]:
# =================================================================================
# Data selection
# =================================================================================

# Select the overall grades by period
sql = "select subject, period, AVG(overall_grade) as median from Grade where period like '%2014.1%' \
        and overall_grade IS NOT NULL and overall_grade <> '' \
        and overall_grade > 0.0 \
        Group by subject"

sql2 = "select subject, period, AVG(overall_grade) as median from Grade where period like '%2014.2%' \
        and overall_grade IS NOT NULL and overall_grade <> '' \
        and overall_grade > 0.0 \
        Group by subject"

sql3 = "select subject, period, AVG(overall_grade) as median from Grade where period like '%2015.1%' \
        and overall_grade IS NOT NULL and overall_grade <> '' \
        and overall_grade > 0.0 \
        Group by subject"

sql4 = "select subject, period, AVG(overall_grade) as median from Grade where period like '%2015.2%' \
        and overall_grade IS NOT NULL and overall_grade <> '' \
        and overall_grade > 0.0 \
        Group by subject"

sql5 = "select subject, period, AVG(overall_grade) as median from Grade where period like '%2016.1%' \
        and overall_grade IS NOT NULL and overall_grade <> '' \
        and overall_grade > 0.0 \
        Group by subject"

sql6 = "select subject, period, AVG(overall_grade) as median from Grade where period like '%2016.2%' \
        and overall_grade IS NOT NULL and overall_grade <> '' \
        and overall_grade > 0.0 \
        Group by subject"

# Select the periods (except 2014.1 because the dataSet isn't complete)
sqlPeriod = "select distinct period from Grade where period not Like '%2014.1%'"

# Execute query and store records in DataFrame: data
data = pd.read_sql_query(sql, engine)
data2 = pd.read_sql_query(sql2, engine)
data3 = pd.read_sql_query(sql3, engine)
data4 = pd.read_sql_query(sql4, engine)
data5 = pd.read_sql_query(sql5, engine)
data6 = pd.read_sql_query(sql6, engine)
dataPeriod = pd.read_sql_query(sqlPeriod, engine)

# =================================================================================
# Chart plotting
# =================================================================================

# Tools
TOOLS = 'box_zoom,box_select,crosshair,resize,reset,hover,save'

# (dict, OrderedDict, lists, arrays and DataFrames are valid inputs)
period = np.array(dataPeriod);

values = np.array([data2['median'], data3['median'], data4['median'], 
                   data5['median'], data6['median']])

# Make a line chart with the dataSet
line = Line(data=values, legend="top_left", background_fill_color="#E8DDCB", 
            ylabel='Overall grades', tools=TOOLS, plot_width=900)

# Configure visual properties on a plot's title attribute
line.title.text = "Overall grades historic chart"
line.title.align = "center"
line.title.text_font_size = "25px"

# Call the output_notebook() 
output_notebook()

# Display the plot
show(line)

Loading BokehJS ...

## 7) Overall students situation

In [48]:
from bokeh.charts import Bar, output_notebook
from bokeh.models import Legend, LegendItem
from bokeh.models import ColumnDataSource, ranges, LabelSet

# =================================================================================
# Data selection
# =================================================================================

# Select the subject list status
sql = "select subject, situation from Grade"

# Execute query and store records in DataFrame: data
data = pd.read_sql_query(sql, engine)

# Create an specific array to subject status
subjectStatus = data.copy()
subjectStatus["Count"] = 0

# Count the status sum-up
subjectStatus = pd.DataFrame(subjectStatus.groupby(["situation"])['Count'].count()).reset_index()

# =================================================================================
# Chart plotting
# =================================================================================

# Tools
TOOLS = 'box_zoom,box_select,crosshair,resize,reset,hover,save'

# Bar chart creation
p = Bar(subjectStatus, values='Count', label='situation', color='situation',
          tools=TOOLS, plot_width=900, plot_height=750)

# Configure visual properties on a plot's title attribute
p.title.text = "Overall students situation"
p.title.align = "center"
p.title.text_font_size = "25px"
p.ylabel = 'Situation frequency'
p.xlabel = 'Situation'
p.legend.location = "top_right"

# Add hover selection for each bar
hover = p.select(dict(type=HoverTool))
hover.tooltips = [('Situation name',' $x'),('Frequency',' $y')]

# Call the output_notebook()
output_notebook()
        
# Plota o grafico
show(p)

Loading BokehJS ...